<a href="https://colab.research.google.com/github/mahalakshmijinadoss/pooling/blob/main/trac2ahin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece]==4.10.0 datasets --quiet


     |████████████████████████████████| 2.8 MB 13.9 MB/s 
     |████████████████████████████████| 311 kB 33.3 MB/s 
     |████████████████████████████████| 596 kB 7.5 MB/s 
     |████████████████████████████████| 3.3 MB 33.8 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 36.4 MB/s 
     |████████████████████████████████| 1.1 MB 36.4 MB/s 
     |████████████████████████████████| 243 kB 46.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.5 MB/s 
     |████████████████████████████████| 133 kB 51.2 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 53.0 MB/s 
     |████████████████████████████████| 144 kB 54.9 MB/s 


In [2]:
!python -c  "import transformers;print(transformers.__version__)"


4.10.0


In [3]:
!pip install -qq ipython-autotime

%load_ext autotime

time: 208 µs (started: 2022-02-22 03:58:26 +00:00)


In [4]:
import torch
torch.cuda.empty_cache()


time: 1.58 s (started: 2022-02-22 03:58:36 +00:00)


In [5]:
import numpy as np
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

time: 388 ms (started: 2022-02-22 03:58:42 +00:00)


In [6]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

time: 6.71 s (started: 2022-02-22 03:58:44 +00:00)


In [7]:
import pandas as pd
import numpy as np


time: 1.61 ms (started: 2022-02-22 03:58:59 +00:00)


In [8]:
from transformers import DataCollatorWithPadding


time: 265 ms (started: 2022-02-22 03:59:00 +00:00)


In [9]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "rohanrajpal/bert-base-en-hi-codemix-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/832 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

time: 5.84 s (started: 2022-02-22 03:59:17 +00:00)


In [11]:
train= pd.read_csv('/content/trac2_hin_train.csv', sep=',')

time: 24.3 ms (started: 2022-02-22 03:59:26 +00:00)


In [12]:
train.shape


(3984, 4)

time: 5.03 ms (started: 2022-02-22 03:59:28 +00:00)


In [13]:
train.head() 


,ID,Text,Sub-task A,Sub-task B
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,NAG,NGEN
1,C4.638,Chutiya movie...,NAG,NGEN
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,OAG,NGEN
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,OAG,NGEN
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


time: 29.4 ms (started: 2022-02-22 03:59:30 +00:00)


In [14]:
train.rename(columns = {'Sub-task A':'label'}, inplace = True)


time: 3.88 ms (started: 2022-02-22 03:59:34 +00:00)


In [17]:
train['label'].value_counts()


0    2245
1    1739
Name: label, dtype: int64

time: 18 ms (started: 2022-02-22 03:59:41 +00:00)


In [16]:
train.loc[(train.label == 'NAG'), 'label'] = 0
train.loc[(train.label == 'CAG'), 'label'] = 1
train.loc[(train.label == 'OAG'), 'label'] = 1

time: 19.7 ms (started: 2022-02-22 03:59:39 +00:00)


In [18]:
dev= pd.read_csv('/content/trac2_hin_dev.csv')


time: 10.4 ms (started: 2022-02-22 03:59:44 +00:00)


In [19]:
dev.rename(columns = {'Sub-task A':'label'}, inplace = True)


time: 2.76 ms (started: 2022-02-22 03:59:47 +00:00)


In [20]:
dev

,ID,Text,label,Sub-task B
0,C38.9,bkl interviewers kuch jaada hi open minded bnt...,OAG,NGEN
1,C4.1510,Bhaiya shaadi mein zaroor aana movie ka plot j...,NAG,NGEN
2,C19.95,Section 375 hai kya??? .... Ye to batate kam s...,NAG,NGEN
3,C4.281,कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...,OAG,NGEN
4,C4.216,Maine itni kam dislike kbhi nhii dekhi,NAG,NGEN
...,...,...,...,...
992,C29.18,Are bhai konsi duniya mai rehate ho😂😂,NAG,NGEN
993,C9.225,Sahi to bola,NAG,NGEN
994,C7.1685,Or tum Kay Kar raya ho?,NAG,NGEN
995,C36.692.1,Ye bhaat sahi hain par zyada dhin thine ga nah...,NAG,NGEN


time: 24.9 ms (started: 2022-02-22 03:59:50 +00:00)


In [21]:
dev.loc[(dev.label == 'NAG'), 'label'] = 0
dev.loc[(dev.label == 'CAG'), 'label'] = 1
dev.loc[(dev.label == 'OAG'), 'label'] = 1

time: 5.18 ms (started: 2022-02-22 03:59:52 +00:00)


In [22]:
import re

time: 1.2 ms (started: 2022-02-22 03:59:54 +00:00)


In [23]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

time: 5.19 ms (started: 2022-02-22 03:59:56 +00:00)


In [24]:
X_train=train.Text.values
X_dev=dev.Text.values
y_train=train.label.values
y_dev=dev.label.values

time: 4.21 ms (started: 2022-02-22 03:59:59 +00:00)


In [25]:
train_texts= np.array([preprocess(text) for text in X_train])
dev_texts= np.array([preprocess(text) for text in X_dev])
num_labels=2


time: 90.1 ms (started: 2022-02-22 04:00:01 +00:00)


In [26]:
train_texts=train_texts.tolist()
dev_texts=dev_texts.tolist()

time: 16.4 ms (started: 2022-02-22 04:00:04 +00:00)


In [27]:
train_texts[0:20]

['Bollywood film dekhne ke samay logic ghar mein chorke ana parta hain.  \nPlease logic mat ghusao',
 'Chutiya movie...',
 'Us jaat bnde ka khene ka matlab tha mar daluga use but gand fat gyi camera\ndekh ke chutiya sala shkl se hi frustrated lag rha hai for sure ye bnda andar\nhi andar gay hoga but iski andar ita dum nahi hoga ki confess kre tabhi sala\nage bhi apne bacho mai khaouf fila rha hai ..chutiya kahinka',
 '@user Is CANCER *un feminist yeh sahi hai  \nbut iska matlab ye thodi na hai ki saare women ko gaali de  \nagar ek insaan ne rape kiya to uske wajh se hume v rapist sunna parega kya?',
 'Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने',
 '@user Chauhan abey tum jse aadmiyo ko ye lgta hai to yhi shi..  \nWse tumhare liye jaruri bhi hai..',
 'Kasturi is a jaahil',
 'isliya jat jhant hota hai...."main jat hu".....use ye nehi pata Baki duniya\nuse jat nehi jhant bolte hai',
 'Lit sir🔥bhenco desh bhakti mai bhi item song pelenge 🤣😂',
 'आर्मी में हमे मर्द चाहिए,हिजड़े नही ।',
 'bro

time: 6.01 ms (started: 2022-02-22 04:00:05 +00:00)


In [28]:
train_encodings = tokenizer(train_texts,truncation=True,)
dev_encodings = tokenizer(dev_texts,truncation=True)

time: 591 ms (started: 2022-02-22 04:00:11 +00:00)


In [29]:
import torch

class TaskDatasetB(torch.utils.data.Dataset):

  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 6.25 ms (started: 2022-02-22 04:00:14 +00:00)


In [30]:
train_dataset = TaskDatasetB(train_encodings,y_train)
dev_dataset = TaskDatasetB(dev_encodings,y_dev)

time: 1.56 ms (started: 2022-02-22 04:00:16 +00:00)


In [31]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


time: 1.56 ms (started: 2022-02-22 04:00:18 +00:00)


In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'hin-trac2',
    run_name = "hi-2epoch",
    evaluation_strategy = 'epoch',
    save_strategy='epoch',
    num_train_epochs = 2,
    load_best_model_at_end = True,
    push_to_hub='hin-trac2'
)

time: 82.5 ms (started: 2022-02-22 04:00:20 +00:00)


In [33]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions  = np.argmax(logits,axis=-1)

  acc = accuracy_score(labels,predictions)
  precision = precision_score(labels,predictions,average='weighted')
  recall = recall_score(labels,predictions,average='weighted')
  f1 = f1_score(labels,predictions,average='weighted')
  
  return {
     'acc' : acc,
     'precision' : precision,
     'recall' : recall,
     'f1' : f1
  }

time: 17.2 ms (started: 2022-02-22 04:00:22 +00:00)


In [34]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 2,ignore_mismatched_sizes=True)


Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rohanrajpal/bert-base-en-hi-codemix-cased and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 24.7 s (started: 2022-02-22 04:00:37 +00:00)


In [36]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Maha/hin-trac2 into local empty directory.


time: 22.4 s (started: 2022-02-22 04:01:08 +00:00)


In [37]:
trainer.train()


***** Running training *****
  Num examples = 3984
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 996


Epoch,Training Loss,Validation Loss,Acc,Precision,Recall,F1
1,No log,0.528359,0.745236,0.774224,0.745236,0.746095
2,0.589500,0.514248,0.780341,0.781671,0.780341,0.780829


***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
Saving model checkpoint to hin-trac2/checkpoint-498
Configuration saved in hin-trac2/checkpoint-498/config.json
Model weights saved in hin-trac2/checkpoint-498/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
Saving model checkpoint to hin-trac2/checkpoint-996
Configuration saved in hin-trac2/checkpoint-996/config.json
Model weights saved in hin-trac2/checkpoint-996/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from hin-trac2/checkpoint-996 (score: 0.5142484307289124).


TrainOutput(global_step=996, training_loss=0.5158406452960279, metrics={'train_runtime': 423.5404, 'train_samples_per_second': 18.813, 'train_steps_per_second': 2.352, 'total_flos': 397310026924320.0, 'train_loss': 0.5158406452960279, 'epoch': 2.0})

time: 7min 3s (started: 2022-02-22 04:02:20 +00:00)


In [38]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 997
  Batch size = 8


{'epoch': 2.0,
 'eval_acc': 0.7803410230692076,
 'eval_f1': 0.7808285156742326,
 'eval_loss': 0.5142484307289124,
 'eval_precision': 0.7816712080565635,
 'eval_recall': 0.7803410230692076,
 'eval_runtime': 13.5314,
 'eval_samples_per_second': 73.68,
 'eval_steps_per_second': 9.238}

time: 13.5 s (started: 2022-02-22 04:09:30 +00:00)


In [39]:
import torch
torch.cuda.empty_cache()

time: 646 ms (started: 2022-02-22 04:09:52 +00:00)


In [40]:
model_path = 'hin-trac2',



time: 1.04 ms (started: 2022-02-22 04:09:55 +00:00)


In [41]:
model.push_to_hub('hin-trac2')


Configuration saved in hin-trac2/config.json
Model weights saved in hin-trac2/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.40k/679M [00:00<?, ?B/s]

Upload file runs/Feb22_04-00-20_e4061e755547/1645502540.7239952/events.out.tfevents.1645502540.e4061e755547.82…

Upload file runs/Feb22_04-00-20_e4061e755547/events.out.tfevents.1645502984.e4061e755547.82.2: 100%|##########…

Upload file runs/Feb22_04-00-20_e4061e755547/events.out.tfevents.1645502540.e4061e755547.82.0:  75%|#######5  …

To https://huggingface.co/Maha/hin-trac2
   8908e3e..8563d2f  main -> main



'https://huggingface.co/Maha/hin-trac2/commit/8563d2fbff3001fee467f3b6e1b5464083fb1388'

time: 10min 22s (started: 2022-02-22 04:09:57 +00:00)


In [42]:
tokenizer.push_to_hub('hin-trac2')


tokenizer config file saved in hin-trac2/tokenizer_config.json
Special tokens file saved in hin-trac2/special_tokens_map.json
To https://huggingface.co/Maha/hin-trac2
   8563d2f..a21e7b8  main -> main



'https://huggingface.co/Maha/hin-trac2/commit/a21e7b810415a5acf054a52587f8651f88883205'

time: 7.55 s (started: 2022-02-22 04:20:45 +00:00)
